# Problem set 5: Clustering (solution)

## Description

**The goal of this problem set is to compute and explore three different clusterings of historical court records.**

## Imports and setup

In [ ]:
%matplotlib inline
import numpy as np
import os
from   sklearn.cluster import KMeans, SpectralClustering, DBSCAN, OPTICS, AgglomerativeClustering
from   sklearn.datasets import make_blobs
from   sklearn.feature_extraction.text import TfidfVectorizer
from   sklearn.metrics.pairwise import cosine_distances, cosine_similarity

# Our input text file
old_bailey_file = os.path.join('..', '..', 'data', 'old_bailey', 'old_bailey.txt')

## Convenience functions

Creating visualizations of your data and pulling sample texts for individual comparison are important parts of assessing the performance of your model. To make these two tasks easier, I've provided a function to do each one.

Note that the visualization function performs dimension reduction, so that we can plot high-dimensional text data in 2-D space. The clustering operations are performed on the original, high-dimensional data.

In [ ]:
# Plotting function
def plot_compare(X, labels, title, reduce=True, alpha=0.2):
    '''
    Takes an array of object data, a set of cluster labels, and a title string
    Reduces dimensions to 2 and plots the clustering.
    Returns nothing.
    '''
    import matplotlib.pyplot as plt
    import seaborn as sns
    from   sklearn.decomposition import TruncatedSVD

    if reduce:
        # TruncatedSVD is fast and can handle sparse inputs
        # PCA requires dense inputs; MDS is slow
        coordinates = TruncatedSVD(n_components=2).fit_transform(X)
    else:
        # Optionally handle 2-D inputs
        coordinates = X
    
    # Set up figure
    fig, ax = plt.subplots(figsize=(12,6))

    # Unlabeled data
    plt.subplot(121) # 1x2 plot, position 1
    plt.scatter(
        coordinates[:, 0], 
        coordinates[:, 1], 
        alpha=alpha, # Set transparency so that we can see overlapping points
        linewidths=0 # Get rid of marker outlines
    )
    plt.title("Unclustered data")

    # Labeled data
    plt.subplot(122)
    sns.scatterplot(
        x=coordinates[:, 0], 
        y=coordinates[:, 1],
        hue=labels,
        alpha=alpha,
        palette='viridis',
        linewidth=0
    )
    plt.title(title)
    plt.show()
    
# Pull sample texts from each label set
def pull_samples(texts, labels, n=3):
    '''
    Takes lists of texts and an array of labels, as well as number of samples to return per label.
    Prints sample texts belonging to each label.
    '''
    texts_array = np.array(texts) # Make the input text list easily addressable by NumPy
    for label in np.unique(labels): # Iterate over labels
        print("Label:", label)
        sample_index = np.where(labels == label)[0] # Limit selection to current label
        print("Number of texts in this cluster:", len(sample_index), '\n')
        chosen = np.random.choice(sample_index, size=n) # Sample n texts with this label
        for choice in chosen:
            print("Sample text:", choice)
            print(texts_array[choice], '\n') # Print each sampled text
        print("###################################")

## Synthetic example

To see how clustering works in `scikit-learn` (it's really easy!), here's a synthetic example.

Note that `X` and `y` are the conventional labels for an input data array and a vector of labels, respectively. You don't have to use these (and may well want to at least give your variables informative suffixes), but you'll see this notation a lot.

In [ ]:
n_samples = 1500 # this many points per blob
X, y = make_blobs(n_samples=n_samples) # Make data and true labels
y_pred = KMeans(n_clusters=3).fit_predict(X) # Perform clustering

plot_compare(X, y, 'True blobs', reduce=False)
plot_compare(X, y_pred, 'k-Means labels', reduce=False)

That's it! Notice that all we needed was to construct input data, set up a clustering object, and call the `fit_predict` method on the input data.

This is what you'll do for the graded portions to follow.

## Old Bailey records

We'll work with a set of 3,090 short text documents from the Old Bailey, the main criminal court of the city of London. These records are a small subset of the almost 200,000 total digitized records collected by [The Old Bailey Proceedings Online](https://www.oldbaileyonline.org/static/Project.jsp).

Our versions of the records have had most names removed. We need to perform some preprocessing, then vectorize the documents.

Note that the file with which we're working collects all the records into a single document. Individual cases are delimited with two newlines, hence we split on `'\n\n'`.

In [ ]:
# Read cases in as a list of strings
with open(old_bailey_file, 'r') as f:
    bailey = [doc for doc in f.read().split('\n\n')] # split on consecutive newlines
print("Total documents:", len(bailey))

## 0. Pre-reflection (10 points)

Reflect briefly on what you expect to find in the data. What types of cases to you expect the court to have adjudicated? How many of each type might you expect? What might be the important differences between case types? How cleanly do you expect the different cases to be separated?

**Your answer here**

## 1. Vectorize (5 points)

Using the vectorizer defined below, transform the input documents into a TFIDF-weighted document-term matrix. Store your vectorized output in a varaible named `X_bailey`.

Note: This should take one line of your own code!

In [ ]:
# Custome preprocessing to remove escaped characters in input
def pre_proc(x):
    '''
    Takes a unicode string.
    Lowercases, strips accents, and removes some escapes.
    Returns a standardized version of the string.
    '''
    import unicodedata
    return unicodedata.normalize('NFKD', x.replace("\'", "'").replace("\ in\ form", " inform").lower().strip())

# Set up vectorizer
vectorizer = TfidfVectorizer(
    encoding='utf-8',
    preprocessor=pre_proc,
    min_df=2, # Note this
    max_df=0.8, # This, too
    binary=False,
    norm='l2',
    use_idf=True # And this
)

# Your code here


# Get the dimensions of the doc-term matrix
print("Matrix shape:", X_bailey.shape)

In [ ]:
# Freebie: Display a document and its vectorized features
doc = 1000 # Which document to use?
print("A sample document:\n", bailey[doc], '\n')
print("The document's features:\n", sorted(vectorizer.inverse_transform(X_bailey[doc])[0]))

In [ ]:
# Freebie: Check that we've retained gendered pronouns in our vectorization
for pronoun in ['she', 'her', 'hers', 'he', 'him', 'his']:
    print(pronoun, '\t', pronoun in vectorizer.get_feature_names())

## 2. Perform *k*-Means clustering (15 points)

Perform *k*-Means clustering with `n_clusters=3` on your vectorized data. See the synthetic example above for guidance and/or consult the scikit-learn [kMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) documentation.

Specifically, you must:

1. Perform the clustering and store your output labels in variable named `y_kmeans`
1. Print the shape of your label vector (it should match the number of documents in the input data set)
1. Plot the resulting clustering using the supplied `plot_compare` function

You should be able to accomplish these tasks in a few lines of code.

In [ ]:
# Perform k-Means clustering with n_clusters = 3


In [ ]:
# Plot k-Means results

## 3. Analyze *k*-Means clustering results (15 points)

Use the supplied `pull_samples` function to examine 5 documents from each cluster (5 points). Then, write a paragraph analyzing your results (10 points). 

Can you make sense of the clusters? Do the cases look like what you expected? Do the calculated clusters strike you as meaningfully distinct? Do you think justice was done in the cases you examined? Anything else that stands out?

In [ ]:
# Pull k-Means samples


**Your discussion of the *k*-Means results here**

## 4. Spectral clustering (30 points)

*k*-Means is just one approach to clustering. Here, you'll produce produce a **Spectral clustering**, with cosine similarity, as a point of comparison. You may want to consult the scikit-learn [SpectralClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html) documentation.

You must:

1. Set up a `SpectralClustering` object with `n_clusters=3` and `affinity='precomputed'`
1. Calculate the pairwise `cosine_similarity` matrix on your vectorized input data
1. Compute a spectral clustering on the cosine similarity matrix, storing the output labels in a variable named `y_spectral`
1. Print the shape of your output label vector
1. Plot your results using the `plot_compare` function
1. Pull and print 5 sample cases belonging to each cluster using the `pull_samples` function
1. Discuss the results of your spectral clustering, both on their own and in comparison to the *k*-Means results. About an honest aragraph should be sufficient.

In [ ]:
# Your spectral clustering code


In [ ]:
# Plot spectral results


In [ ]:
# Pull spectral samples


**Your discussion of the spectral clustering results here**

## 5. Any other clustering method (25 points)

Perform the same steps as in the *k*-Means and Spectral clustering cases (set up with appropriate options, compute labels, print label vector dimensions, plot, pull samples, and discuss results), but using a different clustering method from among [those offered by scikit-learn](https://scikit-learn.org/stable/modules/clustering.html#clustering). Say something brief about why you chose the method you did as part of your discussion.

Note: You'll want to pay particular attention to data input formats and to any key parameters of your chosen clustering method. Just as Spectral clustering required an affinity (similarity) matrix as input, some methods want a precomputed distance or similarity matrix as input. Others are very sensitive to options (like epsilon in DBSCAN). Work with care and note in your discussion any choices that you feel were important.

In [ ]:
# Your code here

**Your discussion here**